In [2]:
import os
from  dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)

True

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'files/chroma/'

In [16]:
embedding = OpenAIEmbeddings()
vector_db = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [17]:
vector_db.delete_collection()

In [7]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [8]:
smalldb = Chroma.from_texts(texts,embedding=embedding)

In [9]:
question = "Tell me about all white mushrooms with large fruiting bodies"

In [11]:
smalldb.similarity_search(question,k=2)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.', metadata={}),
 Document(page_content='The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).', metadata={})]

In [10]:
smalldb.max_marginal_relevance_search(question,k=2,fetch_k=3)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.', metadata={}),
 Document(page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.', metadata={})]

In [18]:
from langchain.document_loaders import PyPDFLoader

loaders = [
    PyPDFLoader("files/cs229/CS229_Note_Apr_8,_2020_(Linear_Regression_&_Normal_Equation).pdf"),
    PyPDFLoader("files/cs229/CS229_Note_Apr_8,_2020_(Linear_Regression_&_Normal_Equation).pdf"),
    PyPDFLoader("files/cs229/CS229_Note_Apr_13,_2020_(Locally_Weighted_Regression_&_Logistic_Regression_&_Newton's_Method).pdf"),
    PyPDFLoader("files/cs229/CS229_Note_Apr_20,_2020_(Generative_Learning_Algorithms_&_Naive_Bayes).pdf"),
    PyPDFLoader("files/cs229/CS229_Note_Apr_22,_2020_(Laplace_Smoothing_&_Kernel_Methods).pdf"),
    PyPDFLoader("files/cs229/CS229_Note_Apr_27,_2020_(Kernel_Methods_&_SVM).pdf"),
    PyPDFLoader("files/cs229/CS229_Note_Apr_29,_2020_(Neural_Networks).pdf"),
]

docs = []
for loader in loaders:
    docs.extend(loader.load())

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=150)

In [6]:
splits = text_splitter.split_documents(docs)

In [7]:
chroma_dir = 'files/chromadb'

In [19]:
vector_db = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=chroma_dir
)

In [20]:
question = 'what did they say about linear regression in the lectures?'

In [21]:
vector_db.max_marginal_relevance_search(question,k=3)

[Document(page_content="OutlineLinearRegressionrecapLocallyweightedRegressionProbailisticinterpretationLogisticRegressionNewton'sMethodRecapsHousingpriceexamplexyithexamplexnicRytRXoIfiIejT8I0o7xxoax0otOixOzJxOzheheofexamplesDoffeatureshoweoojxjOx.JOIhowtyjLocallyWeightedRegressionParametriclearningalgorithmFitfixedsetofparametersOitodateNonparametriclearningalgorithmofparametersgrowslinearlywithsizeofdataweightsmallxxxxxIrsxToevaluatehatcertainXEweightfunction1LRFit0tominimizexxxxxKIlyOTHjBandwidthReturnOTXII", metadata={'page': 0, 'source': "files/cs229/CS229_Note_Apr_13,_2020_(Locally_Weighted_Regression_&_Logistic_Regression_&_Newton's_Method).pdf"}),
 Document(page_content='OutlineNonlinearmodelNeuralnetworkLinearregressionreviewholeOFbJooyhoaxDyotxoil.bgRunGDorSGDNonlinearmodelKernelhangOttolinearin0butnotXnonlinearinboth0andXexhowJOixze TxyAssumewehaveadatasetndyxdEIRDyERhoRdRCostfuncforexampleidcosyhowtCostfarefortatasetNotetheminimizerwillremaintheJcoIEJcosamewoOptimizeminTCOG

In [22]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [23]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `docs/cs229_lectures/MachineLearning-Lecture01.pdf`, `docs/cs229_lectures/MachineLearning-Lecture02.pdf`, or `docs/cs229_lectures/MachineLearning-Lecture03.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [27]:
document_content_description = "Lecture notes"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

ImportError: Cannot import lark, please install it with 'pip install lark'.

In [28]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [30]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [31]:
# Wrap our vectorstore
llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

In [32]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [33]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

InvalidCollectionException: Collection 530487b3-60b3-40c4-bd95-ca6434fdde96 does not exist.